# Bubbles example

In [ ]:
import mefikit as mf
import numpy as np
import pyvista as pv

rng = np.random.default_rng(seed=123)
pv.set_plot_theme("dark")
pv.set_jupyter_backend("static")

In [ ]:
xmax = 5.0
ymax = 1.0
r = 0.17
nb = 15
nr = 12.5

In [ ]:
nx = int(xmax / r * nr)
ny = int(ymax / r * nr)
print(f"Number of elements : {nx * ny * ny:,}")

In [ ]:
xc = rng.uniform(r, xmax - r, nb)
yc = rng.uniform(r, ymax - r, nb)
zc = rng.uniform(r, ymax - r, nb)
spheres = [mf.sel.sphere([x, y, z], r) for x, y, z in zip(xc, yc, zc)]
sphere_union = spheres[0]
for s in spheres[1:]:
    sphere_union = sphere_union | s

In [ ]:
x = np.linspace(0.0, xmax, nx)
y = np.linspace(0.0, ymax, ny)
volumes = mf.build_cmesh(x, y, y)

In [ ]:
volumes.boundaries().to_pyvista().plot(opacity=0.4)

In [ ]:
inner_bubbles = volumes.select(sphere_union)
interface = inner_bubbles.boundaries()
cracked = volumes.crack(interface)

In [ ]:
cracked.boundaries().to_pyvista().plot(opacity=0.4)

In [ ]:
compos = cracked.connected_components()
fluid = compos[-1]
bubbles = compos[:-1]

In [ ]:
pv.global_theme.color_cycler = "default"
pl = pv.Plotter()
for c in bubbles:
    compo = c.to_pyvista()
    pl.add_mesh(compo)
pl.add_mesh(fluid.boundaries(target_dim=1).to_pyvista())
pl.show()
pv.global_theme.color_cycler = None

In [ ]:
clip1 = mf.sel.bbox([-np.inf] * 3, [np.inf, ymax / 2.0, np.inf])
fluid.select(clip1).to_pyvista().plot()